# Fra dat-fil direkte til parquetfil i prodsonen med saspy
Historisk data hos SSB ligger i arkivet som "flate filer".

At en fil er "flat" betyr at den består av faste posisjoner.


## Flat fil vs. CSV
Hvor en CSV bruker semicolon `;` som skilletegn (og dermed kan ha ulik bredde på radene sine):
```csv
012345;;Ola Nordmann;
345678;Kvinne;Kari Nordmann;
```
Er en flat fil, med faste posisjoner, avhengig av at man feks. bruker mellomrom-tegn for å plassere all tekst i korrekte "tegnkolonner":
```csv
012345      Ola Nordmann \n
345678KvinneKari Nordmann\n
```
Her tar jeg med \n (som tegn for ny linje) for å vise at bak `Ola Nordmann ` er det ett ekstra mellomrom for å fylle ut lengden `Kari Nordmann` tar.
Bredden på hver kolonne er dermed en type *metadata*, som vi trenger for å lese inn faste filer. I tillegg trenger vi *metadata* om *kolonne-datatypene* som vi også ønsker ved innlasting av en CSV (string, character, numeric, int, float osv.).


## Innlesing av flat fil direkte i pandas
I pandas, for å laste inn den flate filen direkte, må vi da vite bredden på hver kolonne:

Her må vi manuelt navngi kolonnene, sette datatype, tolke tomme verdier, og regne på/telle hvor brede alle kolonnene er.
Det som er ekstra kjedelig, er at om vi ser på ett SAS-innlastingsscript som vi kan eksportere fra "gamle datadok i prodsonen", så ligger det info der om dette, datatyper, bredder, navn, hvorfor kan vi ikke bare bruke ett sas-innlast script for å hente inn dat-filene i Python direkte? (Skal bli litt mindre retorisk nedover her altså, lover.)

In [1]:
import pandas as pd
from io import StringIO  # Nødvendig siden vi sender en streng, ikke en filsti til .read_fwf
instring = "112345      Ola Nordmann \n345678KvinneKari Nordmann\n"
df = pd.read_fwf(StringIO(instring),
                 names=['pers_id', 'kjonn', 'navn'],  # Navngi kolonner
                 dtype='object',  # Alle kolonnene settes til "object"
                 na_values=['.', ' .'],  # Hvilke karakterer bruker SAS for tom verdi?
                 widths=[6, 6, 13])  # Tell/regn ut dissa sjøl
df

,pers_id,kjonn,navn
0,112345,NaN,Ola Nordmann
1,345678,Kvinne,Kari Nordmann


Her må vi manuelt 
- navngi kolonnene
- sette datatype
- tolke tomme verdier
- og regne på/telle hvor brede alle kolonnene er.

Om du kun skal lese inn en fil, og ikke er motstander av å legge inn dette, så holder dette kanskje? Men om du vil ha en mer programmatisk tilnærming, les videre.

## Har vi ikke info om bredder og datatyper i SAS-innlastingsscriptene allerede?

Det som er ekstra kjedelig, er at om vi ser på ett SAS-innlastingsscript som vi kan eksportere fra "gamle datadok i prodsonen", så ligger det info der om dette: datatyper, bredder og navn, hvorfor kan vi ikke bare bruke ett sas-innlast script for å hente inn dat-filene i Python direkte?

In [2]:
script = """
DATA sas_data;
   INFILE '/ssb/bruker/felles/flatfileksempel_dapla_manual_blogg.dat' MISSOVER PAD LRECL=36;
   INPUT
      @1 pers_id 6.0
      @7 kjonn $CHAR6.0
      @13 navn $CHAR13.0
   ;
RUN;
"""

I teorien kunne man ekstrahert tall og slikt fra dette scriptet, og sendt det inn i `pd.read_fwf()`, men er ikke det også en relativ stor jobb å ta? 

Og hva når man kommer til flyt-tall og det er punktumer i flatfila, og begge tallene på slutten av linja er i bruk, typ: `6.3`.
Er det 9 tegn, 10 inkludert punktumet? <br />Visste du at SAS også har valide datotyper? Dette er jeg for lat til å finne ut av, det må finnes en enklere måte?

## Last innlastingsscriptet direkte fra en fil og inn i en variabel i Python
Innlastingsscriptene er ikke encodet med UTF8 (slik Python forventer), så vi må spesifisere encodingen ved innlast:

In [3]:
sas_script_path = "/ssb/bruker/felles/flatfileksempel_dapla_manual_blogg.sas"
with open(sas_script_path, "r", encoding="latin1") as sas_script:
    script = sas_script.read().strip()
    script = "DATA " + script.split("DATA ")[1] # Forkort ned scriptet til det vi trenger
print(script)

DATA sas_data;
   INFILE '/ssb/bruker/felles/flatfileksempel_dapla_manual_blogg.dat' MISSOVER PAD LRECL=36;
   INPUT
      @1 pers_id 6.0
      @7 kjonn $CHAR6.0
      @13 navn $CHAR13.0
   ;
RUN;


Nå ligger innholdet i sas-scriptet som en streng i variabelen `script`.

## Sett opp saspy med kryptert passord
Når vi nok uansett er i prodsonen med datfilene våre, så kan vi jo utnytte at sas-servene ligger der.

**Resten av oppskriften forutsetter derfor at det kjøres i prodsonen.** 

Saspy er en pakke skrevet av SAS-folka sjøl, med den kan vi snakke med en sas-server vi har tilgang på, fra python.

I pakken `ssb-fagfunksjoner`har vi gjort noen forenklinger på toppen av dette, som lar deg lagre passord i brukermappen din ol.
Innstaller pakken med `poetry add ssb-fagfunksjoner`. (Husk å kjøre `ssb-project build` etterpå, for å få pakken inn i notebookene dine.)

Om du aldri har brukt pakken før, kjør denne funksjonen og følg instruksen for å lagre passordet ditt:

In [4]:
from fagfunksjoner.prodsone import saspy_ssb
help(saspy_ssb.set_password)
#saspy_ssb.set_password("")  # Bruk denne linjen etter å ha lest og fulgt instruksjonen

Help on function set_password in module fagfunksjoner.prodsone.saspy_ssb:

set_password(password: str)
    Pass into this function, an encrypted version of your password that you can get
    in SAS EG, running the following code (swap MY PASSWORD for your actual common-password):
    
    proc pwencode in='MY PASSWORD' method=sas004;
    run;
    
    In the log-window in SAS EG you should then recieve an encrypted version of your password,
    that looks something like this {SAS004}C598BA0A77F74607464634566CCD0D7BB8EBDEEA4B73C440
    Send this as the parameter into this function.
    
    Parameters
    ----------
    password: str
        Your password encrypted using SAS EG



## Send innlastscriptet til SAS-serveren med saspy og få en pandas DataFrame tilbake
Når du har ett passord lagret fra SAS EG, så kan du konstruere en "sas session", dette er en tilkobling til sas-serverene:

In [5]:
from fagfunksjoner import saspy_session
sas = saspy_session()  # Fagfunksjoner-pakken hjelper deg med å lage en tilkobling
result = sas.submit(script)  # Vi bruker tilkoblingen, og sender inn scriptet vi lastet tidligere
log = result["LOG"]  # Du kan se på loggen om du vil
df_frasas = sas.sd2df("sas_data", "work")  # Be sas-serveren om å sende tabellen den har lastet med scriptet, tilbake som en dataframe
sas._endsas()  # La oss være snille med sas-serverene og faktisk lukke tilkoblingen når vi har det vi trenger
df_frasas

SAS Connection established. Subprocess id is 9967

SAS Connection terminated. Subprocess id was 9967


,pers_id,kjonn,navn
0,112345.0,NaN,Ola Nordmann
1,345678.0,Kvinne,Kari Nordmann


### Vurder kolonne-datatyper
Vi har nå en pandas dataframe med datatyper, men disse er basert på den lave mengden datatyper i SAS.
- Character mappes gjerne til "object" i pandas, ikke den strengere varianten "string" (tok du 'an?)
- Numeric mappes stort sett til "float64" i pandas, vi får som regel ikke heltall direkte "Int64"

In [6]:
df_frasas.dtypes

pers_id    float64
kjonn       object
navn        object
dtype: object

Om du vil la pandas gjøre ett nytt forsøk på å gjette på datatyper kan du gjøre noe som dette:

In [7]:
df_pd_dtypes = df_frasas.convert_dtypes()
df_pd_dtypes.dtypes

pers_id             Int64
kjonn      string[python]
navn       string[python]
dtype: object

Om du vil teste min selvskrevne funksjon for å gjette på datatyper så ligger den i fellesfunksjons-pakken:

In [8]:
from fagfunksjoner import auto_dtype
df_auto = auto_dtype(df_frasas)
df_auto.dtypes


Memory usage before cleanup:
378

Memory usage after cleanup:
396
104 % of original size.


pers_id             Int32
kjonn      string[python]
navn       string[python]
dtype: object

Legg merke til at med scriptet så er heltallet enda mindre enn over. Æ gjør en del litt kule ting sjø.


Sjekk gjerne ut parameteret `cardinality_threshold`, om du er interessert i å automatisk sette `categorical dtypes`.

## Lagre dataframen til parquet
Null problem dette nå, når datasettet allerede finnes i pandas.

In [9]:
df_auto.to_parquet("/ssb/bruker/felles/flatfileksempel_dapla_manual_blogg.parquet")

## Videre lesning
I omleggingen av NUDB (Nasjonal utdanningsdatabase), måtte vi konvertere hele arkivet vårt på 750+ dat-filer.

Det var ønskelig å slippe å lagre til sas7bdat i mellom, for å slippe mye dataduplikasjon og arbeidsprosesser.

I stor grad kunne dette arbeidet automatiseres (bortsett fra å lagre ut innlastingsscript fra gamle datadok). Funksjonene jeg utviklet for dette ligger stort sett i denne filen:<br />
[github.com/utd-nudb/prodsone/konverter_arkiv/archive.py](https://github.com/statisticsnorway/utd-nudb/blob/main/prodsone/konverter_arkiv/archive.py)